In [78]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from Laser import Laser
import datetime

# Import custom functions

In [79]:
def findAngle(x,y,x_last,y_last):
    newAngle = np.arctan2((y-y_last),(x-x_last))
    if newAngle < 0:
        newAngle = 2*np.pi - np.abs(newAngle)
    # convert to degrees
    newAngle = newAngle*180/np.pi
    return newAngle

In [80]:
def find_closest_angle(target_angle, anglesObj):
    angles = anglesObj.angles
    # Convert all angles to be in the range [0, 360)
    normalized_angles = np.mod(angles, 360)
    target_angle = target_angle % 360

    # Calculate the distance between the target angle and each angle
    distances = np.minimum(np.abs(target_angle - normalized_angles), 360 - np.abs(target_angle - normalized_angles))

    # Find the index of the smallest distance
    closest_index = np.argmin(distances)

    # Calculate the delta between the target angle and the closest angle
    delta = normalized_angles[closest_index] - target_angle

    n = angles.shape[0]

   
#######
    if delta > 180:
        delta -= 360
    elif delta < -180:
        delta += 360
#######

    if (delta > (360/n)):
        print(f'ERROR in delta calculation! delta: {delta}; index {closest_index}; normalizedAbgle {normalized_angles[closest_index]}')



    # update angles
    anglesObj.update_angles(delta)    

    return closest_index, delta

In [81]:
# def function to loop thru array of coords
def loopCoords(coords, laserObj):
    deltaErr = 0
    # split coords
    X = [x[0] for x in coords]
    Y = [x[1] for x in coords]
    Z = [x[2] for x in coords]
    
    angles = []

    for i in range(len(X)-1):
        x = X[i]
        x_last = X[i-1]
        y = Y[i]
        y_last = Y[i-1]


        angle = findAngle(x,y,x_last,y_last)
        angles.append(angle)

    angles = np.array(angles)

    # find closest angle
    closest_angles = []
    deltas = []
    for i in range(len(angles)):
        closest_angle, delta = find_closest_angle(angles[i], laserObj)
        closest_angles.append(closest_angle)
        deltas.append(delta)
        # collect errors
        if delta > (360/laserObj.angles.shape[0]):
            deltaErr += 1

    return angles, closest_angles, deltas, deltaErr
   

In [82]:
# load test data
p = np.load('npOutput.npy')
print(p.shape)

(1001, 3)


In [83]:
# init lasers
nLasers = 4
Laser = Laser(nLasers)

# get coords
coords = np.array(p)

# run thru loop
angles, closest_angles, deltas, error = loopCoords(coords, Laser)

#print(f'Errors in delta calculation: {error}; Total laser movements: {Laser.absLaserMovement}')

In [84]:
# safe all data as npy in the LogFiles folder, but create a new subfolder for each run, foldernames are an incrementing number
# check highest number in LogFiles folder
l = np.array(os.listdir('LogFiles'))
# if folder is not empty
if l.shape[0] > 0:
    n = np.argmax(l)
    n = l[n]
    print(n)
else:
    n = 0
print(l)

# create new folder
newFolder = 'LogFiles/' + str(int(n)+1)
print(newFolder)
os.mkdir(newFolder)

# save data
np.save(newFolder + '/angles.npy', angles)
np.save(newFolder + '/closest_angles.npy', closest_angles)
np.save(newFolder + '/deltas.npy', deltas)

# save error, total laser movement and nLasers as json file
with open(newFolder + '/Info.json', 'w') as f:
    f.write('{\n')
    f.write(f'\t"error": {error},\n')
    f.write(f'\t"totalLaserMovement": {Laser.absLaserMovement},\n')
    f.write(f'\t"nLasers": {nLasers}\n')
    f.write('}')

print(f'Error abs: {error}; Error rel: {error/np.shape(coords)[0]}')

8
['.DS_Store' '7' '6' '8' '4' '2' '5']
LogFiles/9
Error abs: 0; Error rel: 0.0


In [85]:
# safe the parameters into a csv file for later comparison
# open log file
logFile = open('logFile.csv', 'a')
# write header only if file is empty
if os.stat('logFile.csv').st_size == 0:
    logFile.write('Datetime;absLaserMovement;nLaser\n')

# write data on new line
logFile.write(f'{datetime.datetime.now()};{Laser.absLaserMovement};{nLasers}\n')



# close log file
logFile.close()




In [86]:
print(deltas)

[-32.7574772531874, -18.85566155249657, 51.613138805683974, 30.166683096437964, -8.933790801018972, -9.146040265199417, -8.926490404173762, -3.156297807376575, -0.8213518846878287, -4.89330507148135, -5.7061015350318645, -1.1495228304446847, 0.43943521367901006, 0.358493357276755, 0.5102346557744237, 0.4737564086833377, 0.47294568978524687, 0.3745207062297009, 0.47450926352239975, 0.42128490416883224, 0.5221654826702817, 0.474527263561745, 0.43977941251607966, 0.4454847220319449, 0.34042883934726564, 0.5090257348996037, 0.45447404845822614, 0.4951108876502266, 0.45545835915252564, 0.4263135913043197, 0.43417486768142055, 0.42176253382835727, 0.41145243046547453, 0.5123109157242531, 0.4316485086946025, 0.5135148457988332, 0.42973278765621714, 0.4442006754048605, 0.42445160894467276, 0.4550197880658741, 0.45501978806589705, 0.4244516089446506, 0.44420067540488617, 0.5027428303466195, 0.3708228773552946, 0.5741524959999538, 0.4394888541719979, 0.411452430465431, 0.42176253382854156, 0.434